In [1]:
import os
import re
import json
import chardet
import src.public_function as pf

# NOTE 中间的所有文件都是UTF8
encoding = 'utf16'
output_encoding = 'utf16'

pf._init_()
name = ''

In [ ]:
# 从游戏原文本文件提取日文

def get_scenario_from_origin(data: list) -> list:
    text_all = []
    cnt = 0
    # START FIXME
    for line in data:
        if pf.has_jp(line) and line[0] not in ';[/*\t# ':
            text_all.append(line)
        cnt+=1
    # END
    return text_all

pf.extract_jp(get_scenario_from_origin, encoding)


In [ ]:
pf.convert_code('utf-8',path='chs')

In [ ]:
# 创建翻译字典
pf.create_dict(lambda x:x[:-1])

In [ ]:
# 检查字典key数量是否正确
pf.check_dict()

In [2]:
# 把字典翻译

# print(func('「・・・ああ、うん、それで・・・スペシャル餃子、一つ・・・」'))
# pf.translate(pf.format_sakura)
print(pf._translate('"■１．思い切って提案する\r\n■２．やっぱり無理だ\n"', delete_func=lambda x: x.replace('・・・', '…')))

"■1.下定决心提出建议
■2.果然做不到
"


In [ ]:
# 检查是否还有没翻译的语句
pf.check_dict_untranslated()

In [ ]:
# 构建对比
pf.create_contrast()

In [ ]:
# 替换文本
def find(data:list, failed_text:list, jp_chs:dict):
    record = 0
    cnt = 0                         # 当前行
    start = False
    for line in data:
        #START FIXME 替换目标文本
        if pf.has_jp(line) and line[0] not in ';[/*\t# ':
            key = line[:-1]
            if key in jp_chs and jp_chs[key]:
                record += 1
                data[cnt] = data[cnt].replace(key, jp_chs[key])
            else:
                failed_text.append(key+'\n')
        cnt += 1
        # END 
    return record

pf.replace_all(find, encoding, output_encoding)

In [ ]:
jp_chs = dict()
chs_file = os.listdir('chs')
for f in chs_file:
    _chs = pf.open_json(f'chs/{f}')
    f = f.split('.')[0]+'.json'
    json_root = 'intermediate_file/split_scenario/'+f
    if os.path.exists(json_root):
        _jp = pf.open_json(json_root)
        _key_chs = list(_chs.keys())
        _key_jp = list(_jp.keys())
        if len(_key_chs) == len(_key_jp):
            for i in range(len(_key_jp)):
                jp_chs[_key_jp[i]] = _key_chs
            # print('输入成功：', f)
        else:
            print('中日不匹配：', f)
    else:
        print('文件不存在：', f)
pf.save_json('intermediate_file/jp_chs.json', jp_chs)

In [ ]:
jp_chs = pf.open_json("intermediate_file/jp_chs.json")
for key in jp_chs:
    jp_chs[key] = key
pf.save_json("intermediate_file/jp_chs.json", jp_chs)

In [ ]:
chs_file = os.listdir('chs')
for f in chs_file:
    new_name = f.replace(".txt",".json")
    os.rename(f"CHS/{f}",f"CHS/{new_name}")

In [ ]:
"""
"", ",
: "" : "
([^"]),\n $1",\n
:([^"]) : "$1
: ([^"]) : "$1
"""

jp_chs = pf.open_json("intermediate_file/jp_chs.json")
chs_file = os.listdir('chs')
for f in chs_file:
    # print(f)
    try:
        _chs = pf.open_json(f'chs/{f}')
        for key in _chs:
            jp_chs[key] = _chs[key]
    except Exception as e:
        print(f)

pf.save_json("intermediate_file/jp_chs.json", jp_chs)


In [ ]:
data="隐藏文字,剧本回忆,跳过已读的文章,自动播放文本,保存,加载,设置,自动跳过已读的文章,文本速度,自动文本发送时间设置,字体选择,播放声音,音量调节,BGM,效果音,台词,MIDI输出端口选择,全屏,游戏结束,游戏结束,返回标题,结束"
ans = ' '.join(list(map(lambda x:hex(x)[2:], data.encode('gbk'))))
print(ans, hex(len(ans)))

In [ ]:
'戦国の黒百合～ふたなり姫と隷属の少女～'.encode('gbk')

In [ ]:
os.mkdir(f'chs')

In [ ]:
def extract_med():
    '''
    从input文件夹内的脚本文件抽取文本
    文本自动存入intermediate_file/jp_all.txt        
    '''
    def _has_jp(line: str) -> bool:
        '''
        如果含有日文文字（除日文标点）则认为传入文字含有日文, 返回true
        '''
        for ch in line:
            if ('\u0800' <= ch and ch <= '\u9fa5') or ('\uff01' <= ch <= '\uff5e'):
                return True
        return False

    file_all = os.listdir('input')
    ans = []
    for f in file_all:
        _ans = []
        _data = pf.open_file_b(f'input/{f}')
        # _data = MED.decrypt(_data)
        _offset = int.from_bytes(_data[4:8], byteorder='little') + 0x10
        _str = _data[_offset:]
        _buff = b''
        for i in _str:
            if i:
                _buff += pf.to_bytes(i, 1)
            else:
                try:
                    _buff = _buff.decode('cp932')
                    if _has_jp(_buff) and _buff[0] not in ';#':
                        # if name:
                        #     _buff = _buff.replace('＄０', name)
                        _ans.append(_buff)
                except Exception as e:
                    print(e)
                    print(f, _buff)
                _buff = b''
        _t = {}
        for i in _ans:
            _t[i]=""
        pf.save_file(f"chs/{f}", json.dumps(_t, ensure_ascii=False, indent=2))
extract_med()